In [1]:
import torch
import torch.nn as nn
import os

from tqdm import tqdm
import numpy as np

from scipy.stats import spearmanr
import csv

In [2]:
def read_nodes(file_path):
    int_list = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            for item in row:
                try:
                    int_list.append(int(item))
                except ValueError:
                    print(f"Warning: '{item}' could not be converted to an integer and was skipped.")
    return int_list

def calculate_one(path):
    # score = torch.load(path, map_location=torch.device('cpu'))  # _test_0225_regroup
    score = torch.load(path, map_location=torch.device('cpu'))
    # score = torch.rand(5000, 500)
    print("score shape:", score.shape)

    nodes_str = []
    for i in range(50):
        nodes_str.append(f"./checkpoints/{i}/train_index.csv")

    full_nodes = [i for i in range(4656)]

    node_list = []
    for node_str in nodes_str:
        numbers = read_nodes(node_str)
        index = []
        for number in numbers:
            index.append(full_nodes.index(number))
        node_list.append(index)

    loss_list = torch.load("./result/gt.pt", map_location=torch.device('cpu')).detach()

    approx_output = []
    for i in range(len(nodes_str)):
        score_approx_0 = score[node_list[i], :]
        sum_0 = torch.sum(score_approx_0, axis=0)
        approx_output.append(sum_0)

    print(len(loss_list), loss_list[0].shape)
    print(len(approx_output), approx_output[0].shape)

    res = 0
    counter = 0
    for i in range(score.shape[1]):
        tmp = spearmanr(np.array([approx_output[k][i] for k in range(len(approx_output))]),
                        np.array([loss_list[k][i].numpy() for k in range(len(loss_list))])).statistic
        if np.isnan(tmp):
            print("Numerical issue")
            continue
        res += tmp
        counter += 1

    print(counter)

    return res/counter, loss_list, approx_output


In [3]:
path = "./result/score.pt"
print(calculate_one(path)[0])

/tmp/ipykernel_3899744/1026402471.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  score = torch.load(path, map_location=torch.device('cpu'))


score shape: torch.Size([4, 1])


/tmp/ipykernel_3899744/1026402471.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loss_list = torch.load("./result/gt.pt", map_location=torch.device('cpu')).detach()


IndexError: index 3155 is out of bounds for dimension 0 with size 4

In [4]:
score = torch.load('./result/score_FJLT-512.pt', map_location=torch.device('cpu'))
# score = torch.rand(5000, 500)
print("score shape:", score.shape)
print(score[0:4, 0])

score shape: torch.Size([100, 10])
tensor([20.5490, 28.3675, 27.1684, 22.0018])


/tmp/ipykernel_3899744/1346549292.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  score = torch.load('./result/score_FJLT-512.pt', map_location=torch.device('cpu'))


In [5]:
score = torch.load('./result/score_FJLT-512_threshold-1e-07.pt', map_location=torch.device('cpu'))
# score = torch.rand(5000, 500)
print("score shape:", score.shape)
print(score[0:4, 0])

score shape: torch.Size([100, 10])
tensor([20.5487, 28.3678, 27.1683, 22.0017])


/tmp/ipykernel_3899744/669972308.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  score = torch.load('./result/score_FJLT-512_threshold-1e-07.pt', map_location=torch.devi

In [6]:
score = torch.load('./result/score_FJLT-512_GIP.pt', map_location=torch.device('cpu'))
# score = torch.rand(5000, 500)
print("score shape:", score.shape)
print(score)

score shape: torch.Size([4, 1])
tensor([[0.0656],
        [0.0806],
        [0.0821],
        [0.0708]])


/tmp/ipykernel_3899744/818564332.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  score = torch.load('./result/score_FJLT-512_GIP.pt', map_location=torch.device('cpu'))


In [7]:
score = torch.load('./result/score.pt', map_location=torch.device('cpu'))
# score = torch.rand(5000, 500)
print("score shape:", score.shape)
print(score)

score shape: torch.Size([4, 1])
tensor([[-0.0019],
        [ 0.0029],
        [ 0.0151],
        [ 0.0002]])


/tmp/ipykernel_3899744/2181850493.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  score = torch.load('./result/score.pt', map_location=torch.device('cpu'))


In [8]:
x = np.array([0.0656, 0.0806, 0.0821, 0.708])

x * (0.0338 / 0.0656)

array([0.0338    , 0.04152866, 0.04230152, 0.36479268])